### Mid-term for HDS5210

Your supervisor is concerned about 4-year survival risks for COPD. She has asked for you to do some analysis using a new metric, BODE. BODE is an improvement on a previous metric and promises to provide insight on survival risks.

BODE is defined here. https://www.mdcalc.com/calc/3916/bode-index-copd-survival#evidence

Your assignment is to create a BODE calculation, use it to calculate BODE scores and BODE survival rates for a group of patients. Then we want to evaluate the average BODE scores and BODE survival rates for each area hospital.

Your patient input file will have the following columns:
NAME,SSN,LANGUAGE,JOB,HEIGHT_M,WEIGHT_KG,fev_pct,dyspnea_description,distance_in_meters,hospital

BODE calculations require a BMI value, so you will have to create a function for it.

Your output should be in the form of two CSV files, patient_output.csv and hospital_output.csv.

Patient_output will have the following columns:
NAME,BODE_SCORE,BODE_RISK,HOSPITAL

Hospital output will have the following columns:
HOSPITAL_NAME, COPD_COUNT, PCT_OF_COPD_CASES_OVER_BEDS, AVG_SCORE, AVG_RISK

Each function you create should have documentation and a suitable number of test cases. If the input data could be wrong, make sure to raise a Value Error.

For this assignment, use the doctest, json, and csv libraries. Pandas is not allowed for this assignment.

In [131]:
import doctest
import json
import csv

### Step 1: Calculate BMI

In [126]:
def calculate_bmi(weight_kg: float, height_m: float) -> float:

    """

    Calculate the Body Mass Index (BMI).



    :param weight_kg: Weight of the patient in kilograms

    :param height_m: Height of the patient in meters

    :return: Calculated BMI

    :raises ValueError: if weight or height are non-positive

    """

    if weight_kg <= 0 or height_m <= 0:

        raise ValueError("Weight and height must be positive values.")



    return weight_kg / (height_m ** 2)

### Step 2: Calculate BODE Score

In [122]:
def calculate_bode_score(bmi: float, fev_pct: float, dyspnea_desc: str, walk_distance: int) -> int:

    """

    Calculate the BODE score based on BMI, FEV1%, dyspnea, and walk distance.



    :param bmi: Body Mass Index

    :param fev_pct: FEV1% of predicted

    :param dyspnea_desc: Description of dyspnea (shortness of breath)

    :param walk_distance: Distance covered in the 6-minute walk test in meters

    :return: BODE score (integer between 0 and 10)

    :raises ValueError: if inputs are not valid

    """

    # BMI scoring

    if bmi > 21:

        bmi_score = 0

    else:

        bmi_score = 1



    # FEV1% scoring

    if fev_pct >= 65:

        fev_score = 0

    elif 50 <= fev_pct < 65:

        fev_score = 1

    elif 36 <= fev_pct < 50:

        fev_score = 2

    else:

        fev_score = 3



    # Dyspnea scoring

    dyspnea_map = {

        'None': 0,

        'STOPS AFTER A FEW MINUTES': 1,

        'WHEN HURRYING': 2,

        'BREATHLESS WHEN DRESSING': 3,

        'WALKING UPHILL': 4,

        'ONLY STRENUOUS EXERCISE': 5,

        'STOPS AFTER 100 YARDS': 6,

        'UNABLE TO LEAVE HOME': 7,

        'SLOWER THAN PEERS': 8,

        'STOPS WHEN WALKING AT PACE': 9


    }



    if dyspnea_desc not in dyspnea_map:

        raise ValueError("Invalid dyspnea description.")



    dyspnea_score = dyspnea_map[dyspnea_desc]



    # Walk distance scoring

    if walk_distance >= 350:

        walk_score = 0

    elif 250 <= walk_distance < 350:

        walk_score = 1

    elif 150 <= walk_distance < 250:

        walk_score = 2

    else:

        walk_score = 3



    # Total BODE score

    return bmi_score + fev_score + dyspnea_score + walk_score



### Step 3: Calculate BODE Risk

In [120]:
def bode_risk(bode_score: int) -> float:

    """

    Map the BODE score to a survival risk percentage.



    :param bode_score: The BODE score (0 to 10)

    :return: The survival risk percentage

    """

    if bode_score == 0:

        return 80.0

    elif bode_score <= 2:

        return 70.0

    elif bode_score <= 4:

        return 60.0

    elif bode_score <= 6:

        return 50.0

    elif bode_score <= 8:

        return 30.0

    else:

        return 20.0

### Step 4: Load Hospital Data

In [133]:
def process_patients(input_file: str, beds_per_hospital: dict):

    """

    Process patient data, calculate BODE scores and risks, and generate two CSV files.



    :param input_file: Path to the patient input CSV file

    :param beds_per_hospital: A dictionary mapping hospital names to number of beds

    """

    patients = []

    hospitals = {}



    # Read input file

    with open(input_file, mode='r') as file:

        csv_reader = csv.DictReader(file)

        for row in csv_reader:

            try:

                # Extract and validate input data

                name = row['NAME']

                hospital = row['hospital']

                weight = float(row['WEIGHT_KG'])

                height = float(row['HEIGHT_M'])

                fev_pct = float(row['fev_pct'])

                dyspnea_desc = row['dyspnea_description']

                walk_distance = float(row['distance_in_meters'])



                # Calculate BMI

                bmi = calculate_bmi(weight, height)



                # Calculate BODE score

                bode_score = calculate_bode_score(bmi, fev_pct, dyspnea_desc, walk_distance)



                # Calculate BODE survival risk

                risk = bode_risk(bode_score)



                # Append to patient list

                patients.append({

                    'NAME': name,

                    'BODE_SCORE': bode_score,

                    'BODE_RISK': risk,

                    'HOSPITAL': hospital

                })



                # Aggregate hospital data

                if hospital not in hospitals:

                    hospitals[hospital] = {'count': 0, 'total_score': 0, 'total_risk': 0}



                hospitals[hospital]['count'] += 1

                hospitals[hospital]['total_score'] += bode_score

                hospitals[hospital]['total_risk'] += risk



            except ValueError as e:

                print(f"Error processing patient {row['NAME']}: {e}")



    # Write patient output

    with open('patient_output.csv', mode='w', newline='') as file:

        writer = csv.DictWriter(file, fieldnames=['NAME', 'BODE_SCORE', 'BODE_RISK', 'HOSPITAL'])

        writer.writeheader()

        for patient in patients:

            writer.writerow(patient)



    # Write hospital output

    with open('hospital_output.csv', mode='w', newline='') as file:

        writer = csv.DictWriter(file, fieldnames=['HOSPITAL_NAME', 'COPD_COUNT', 'PCT_OF_COPD_CASES_OVER_BEDS', 'AVG_SCORE', 'AVG_RISK'])

        writer.writeheader()

        for hospital, data in hospitals.items():

            beds = beds_per_hospital.get(hospital, 100)  # Assuming 100 beds if not provided

            writer.writerow({

                'HOSPITAL_NAME': hospital,

                'COPD_COUNT': data['count'],

                'PCT_OF_COPD_CASES_OVER_BEDS': (data['count'] / beds) * 100,

                'AVG_SCORE': data['total_score'] / data['count'],

                'AVG_RISK': data['total_risk'] / data['count']

            })



if __name__ == "__main__":

    # Example dictionary for hospital beds

    beds_per_hospital = {

        "General Hospital": 120,

        "City Clinic": 85,

        "Metro Health": 150

    }



    # Call the function with input file and hospital bed data

    process_patients('patient.csv', beds_per_hospital)

### Step 5: Main business logic

Call BODE Score, BODE Risk functions for each patient.

For each hospital, calculate Avg BODE score and Avg BODE risk and count the number of cases for each hospital.

In [112]:
patient_csv = "patient.csv"
hospital_json = "hospitals.json"

patient_output_file = "patient_output.csv"
hospital_output_file = "hospital_output.csv"

###
# Your logic here
### this logic is alresdy included in above function

    # Example dictionary for hospital beds

beds_per_hospital = {

        "General Hospital": 120,

        "City Clinic": 85,

        "Metro Health": 150

    }
       # Call the function with input file and hospital bed data

process_patients('patient.csv', beds_per_hospital)
###
patient_results = []
hospital_output_list = []

#Write Patient_output.csv
with open(patient_output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(patient_results)
#Write Hospital_output.csv
with open(hospital_output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(hospital_output_list)